## The problem

Say there is a coin that before a long game is set to one of two states, State 1 $p_H=\frac{1}{2}$ or State 2  $p_H = \frac{3}{4}$ . If you do not know to what state the coin was set to before the game but believe that it was set to state 2 with probability $0.9$, how do coin tosses update your belief?


## Code Talks

In [1]:
import numpy as np

In [2]:
def update_belief(old_belief, conditionals, event):
    '''
    Upates the old belief given that the result of 
    a coin toss (event) is available
    '''
    # Make sure the event makes sense
    assert event in ['head','tail'], \
           print('Event must be head or tail')
    
    # This is what is called the prior I think
    P_A = old_belief['fair']
    
    # B is the event
    P_B_A = conditionals[event]['fair']
    
    P_B = old_belief['fair']   * conditionals[event]['fair'] + \
          old_belief['loaded'] * conditionals[event]['loaded']
      
    # This is the posterior  
    P_B_A = P_A * P_B_A / P_B
    
    # Now we update
    new_belief = {'fair': P_B_A,
                  'loaded': 1 - P_B_A}
    
    return new_belief

In [3]:
def test_belief(first_belief, conditionals, n_trials, p_sequence):
    
    current_belief = first_belief.copy()

    fair   = [current_belief['fair']] 
    history = [current_belief]
    
    p = p_sequence
    choices = np.random.choice(['head','tail'], n_trials, p=[p,1-p])
    #print('Choices', choices)
    #print(list(choices).count('head'))
    
    
    for choice in choices:
        #print('Current Belief: ', current_belief)
        #print('Choice: ', choice)
        new_belief = update_belief(current_belief, 
                                   conditionals, 
                                   choice)
        #print('New Belief: ', new_belief)
        
        history.append((choice, new_belief))
        current_belief = new_belief.copy()
        
        fair.append(current_belief['fair'])
        #print()
        
    return {'fair': fair[-1],
            'loaded': 1-fair[-1]}, fair, history

In [4]:
initial_belief = {'fair' : 0.50, 
                  'loaded': 0.50}

conditionals = { 'head': {'fair':   0.5 ,
                          'loaded': 0.55} ,
                 'tail': {'fair':   0.5,
                          'loaded': 0.45}
               }

In [5]:
new_belief, fair, history = test_belief(initial_belief, 
                                        conditionals, 
                                        800, 
                                        0.55)

In [6]:
new_belief

{'fair': 0.008087010498502219, 'loaded': 0.9919129895014978}

## This is just plotting

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Range1d

In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
p = figure( y_axis_type="log")
p.line(list(range(len(fair))),fair)

show(p)